In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

In [2]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

In [3]:
from torchvision.datasets import MNIST

download_root = '/home/ohjungmin/Week5/data'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 11731801.98it/s]


Extracting /home/ohjungmin/Week5/data/MNIST/raw/train-images-idx3-ubyte.gz to /home/ohjungmin/Week5/data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 5888090.89it/s]


Extracting /home/ohjungmin/Week5/data/MNIST/raw/train-labels-idx1-ubyte.gz to /home/ohjungmin/Week5/data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5869017.19it/s]


Extracting /home/ohjungmin/Week5/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/ohjungmin/Week5/data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 33189074.51it/s]


Extracting /home/ohjungmin/Week5/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/ohjungmin/Week5/data/MNIST/raw



In [4]:
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [5]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [6]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias) # x -> h 로 가는 것 
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias) # h -> h 로 가는 것 
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):        
        hx, cx = hidden        
        x = x.view(-1, x.size(1))
        
        gates = self.x2h(x) + self.h2h(hx) # x2h -> weight가 들어가 있는 것이다.     
        gates = gates.squeeze()        
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        
        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)
        
        cy = torch.mul(cx, forgetgate) +  torch.mul(ingate, cellgate)        
        hy = torch.mul(outgate, F.tanh(cy))        
        return (hy, cy)

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
         
        self.layer_dim = layer_dim               
        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)          
        self.fc = nn.Linear(hidden_dim, output_dim)
         
    def forward(self, x):        
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        if torch.cuda.is_available():
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), hidden_dim))
                           
        outs = []        
        cn = c0[0,:,:]
        hn = h0[0,:,:]

        for seq in range(x.size(1)):
            hn, cn = self.lstm(x[:,seq,:], (hn,cn)) 
            outs.append(hn)
                
        out = outs[-1].squeeze()        
        out = self.fc(out) 
        return out

In [8]:
input_dim = 28
hidden_dim = 128
layer_dim = 1  
output_dim = 10
 
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():
    model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
seq_dim = 28 
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  # 훈련 데이터 셋을 이용한 모델 학습         
        if torch.cuda.is_available(): 
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else: # GPU를 사용하지 않기 때문에 ELSE 구문이 실행이 된다.
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)
          
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        if torch.cuda.is_available(): # cuda가 아니면 아닌것에 대해 정의를 해준 것이다. 
            loss.cuda()

        loss.backward()
        optimizer.step()        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 500 == 0:         
            correct = 0
            total = 0
            for images, labels in valid_loader:
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1 , seq_dim, input_dim))
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                 
                total += labels.size(0)
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.237457275390625. Accuracy: 21.420000076293945
Iteration: 1000. Loss: 1.182291030883789. Accuracy: 59.400001525878906
Iteration: 1500. Loss: 0.39404284954071045. Accuracy: 89.33000183105469
Iteration: 2000. Loss: 0.21669703722000122. Accuracy: 93.08999633789062
Iteration: 2500. Loss: 0.12927502393722534. Accuracy: 94.91999816894531
Iteration: 3000. Loss: 0.07725589722394943. Accuracy: 96.05000305175781
Iteration: 3500. Loss: 0.059582021087408066. Accuracy: 96.69999694824219
Iteration: 4000. Loss: 0.055356815457344055. Accuracy: 97.06999969482422
Iteration: 4500. Loss: 0.04483791068196297. Accuracy: 97.01000213623047
Iteration: 5000. Loss: 0.10861039161682129. Accuracy: 96.66999816894531
Iteration: 5500. Loss: 0.12848277390003204. Accuracy: 97.33000183105469
Iteration: 6000. Loss: 0.012510874308645725. Accuracy: 97.5199966430664
Iteration: 6500. Loss: 0.03014259971678257. Accuracy: 97.68000030517578
Iteration: 7000. Loss: 0.01776876300573349. Accuracy: 97.69999694